In [272]:
import pandas as pd
import csv 
import sqlite3
from geopy.geocoders import OpenCage
import requests
import re


In [273]:
nobel_data = "Resources/nobel_latest.csv"

In [274]:
nobel_df = pd.read_csv(nobel_data)
nobel_df

,Year,Laureate_Id,Firstname,Lastname,Category,Gender,Prize_Share,Motivation,Birth_Date,Birth_Country,Birth_City,Birth_Country_Code,Death_Date,Death_Country,Death_City,Death_Country_Code,Organization_Name,Organization_City,Organization_Country
0,1901,1,Wilhelm Conrad,Röntgen,physics,male,1,"""in recognition of the extraordinary services ...",1845-03-27,Prussia (now Germany),Lennep (now Remscheid),DE,10/2/1923,Germany,Munich,DE,Munich University,Munich,Germany
1,1902,2,Hendrik A.,Lorentz,physics,male,2,"""in recognition of the extraordinary service t...",1853-07-18,the Netherlands,Arnhem,NL,4/2/1928,the Netherlands,NaN,NL,Leiden University,Leiden,the Netherlands
2,1902,3,Pieter,Zeeman,physics,male,2,"""in recognition of the extraordinary service t...",1865-05-25,the Netherlands,Zonnemaire,NL,9/10/1943,the Netherlands,Amsterdam,NL,Amsterdam University,Amsterdam,the Netherlands
3,1903,4,Henri,Becquerel,physics,male,2,"""in recognition of the extraordinary services ...",1852-12-15,France,Paris,FR,25-08-1908,France,NaN,FR,École Polytechnique,Paris,France
4,1903,5,Pierre,Curie,physics,male,4,"""in recognition of the extraordinary services ...",1859-05-15,France,Paris,FR,19-04-1906,France,Paris,FR,École municipale de physique et de chimie indu...,Paris,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,2022,1019,Memorial,NaN,peace,org,3,"""The Peace Prize laureates represent civil soc...",1987-00-00,NaN,NaN,NaN,0000-00-00,NaN,NaN,NaN,NaN,NaN,NaN
985,2022,1020,Center for Civil Liberties,NaN,peace,org,3,"""The Peace Prize laureates represent civil soc...",2007-00-00,NaN,NaN,NaN,0000-00-00,NaN,NaN,NaN,NaN,NaN,NaN
986,2022,1021,Ben,Bernanke,economics,male,3,"""for research on banks and financial crises""",13-12-1953,USA,Augusta GA,US,0000-00-00,NaN,NaN,NaN,The Brookings Institution,Washington D.C.,USA
987,2022,1022,Douglas,Diamond,economics,male,3,"""for research on banks and financial crises""",25-10-1953,USA,Chicago IL,US,0000-00-00,NaN,NaN,NaN,University of Chicago,Chicago IL,USA


In [275]:
cleaned_nobel_df = nobel_df.drop(["Prize_Share", "Birth_Country_Code", "Death_Country", "Death_City", "Death_Country_Code",
               "Organization_Name", "Organization_City", "Organization_Country"], axis = 1)

In [276]:
cleaned_nobel_df['Year'] = cleaned_nobel_df['Year'].astype('Int64')
cleaned_nobel_df['Laureate_Id'] = cleaned_nobel_df['Laureate_Id'].astype(str)
cleaned_nobel_df["Name"] = cleaned_nobel_df["Firstname"].astype(str)+ " " + cleaned_nobel_df["Lastname"].astype(str)
cleaned_nobel_df["Birth_Country"] = cleaned_nobel_df["Birth_Country"].astype(str)
cleaned_nobel_df["Birth_City"] = cleaned_nobel_df["Birth_City"].astype(str)

In [277]:
column_to_move = cleaned_nobel_df.pop("Name")

cleaned_nobel_df.insert(2, "Name", column_to_move)

In [278]:
full_name_df = cleaned_nobel_df.drop(["Firstname", "Lastname"], axis= 1)

In [279]:
full_name_df.head()

,Year,Laureate_Id,Name,Category,Gender,Motivation,Birth_Date,Birth_Country,Birth_City,Death_Date
0,1901,1,Wilhelm Conrad Röntgen,physics,male,"""in recognition of the extraordinary services ...",1845-03-27,Prussia (now Germany),Lennep (now Remscheid),10/2/1923
1,1902,2,Hendrik A. Lorentz,physics,male,"""in recognition of the extraordinary service t...",1853-07-18,the Netherlands,Arnhem,4/2/1928
2,1902,3,Pieter Zeeman,physics,male,"""in recognition of the extraordinary service t...",1865-05-25,the Netherlands,Zonnemaire,9/10/1943
3,1903,4,Henri Becquerel,physics,male,"""in recognition of the extraordinary services ...",1852-12-15,France,Paris,25-08-1908
4,1903,5,Pierre Curie,physics,male,"""in recognition of the extraordinary services ...",1859-05-15,France,Paris,19-04-1906


In [280]:
def clean_country_name(place):
    # Find the text inside parentheses
    match = re.search(r'\(([^()]+)\)', place)
    
    if match:
        # Split the text inside parentheses and extract the last word
        words = match.group(1).split()
        if len(words) >= 2:
            country_name = words[-1]
        else:
            country_name = place
    else:
        country_name = place
    
    return country_name

# Apply the function to clean the column
full_name_df['Birth_Country'] = full_name_df['Birth_Country'].apply(clean_country_name)
full_name_df['Birth_City'] = full_name_df['Birth_City'].apply(clean_country_name)

In [281]:
full_name_df['Birth_Country'] = full_name_df['Birth_Country'].replace('USA', 'United States')
full_name_df['Birth_Country'] = full_name_df['Birth_Country'].replace('the Netherlands', 'Netherlands')
full_name_df['Birth_Country'] = full_name_df['Birth_Country'].replace('Canada', 'Canada')

In [282]:
# Mapping dictionary of state abbreviations to full names
state_mapping = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Deleware',
    'DC': 'District of Columbia',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennesee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconson',
    'WY': 'Wyoming'
    }

# Loop through the cities column in your DataFrame
for i, city in enumerate(full_name_df['Birth_City']):
    # Split the city string into city name and state abbreviation
    city_parts = city.split(' ')
    if len(city_parts) == 2:
        city_name, state_abbr = city_parts
        # Check if the state abbreviation exists in the mapping dictionary
        if state_abbr in state_mapping:
            # Replace the state abbreviation with the full state name
            full_name_df.loc[i, 'Birth_City'] = f"{city_name}, {state_mapping[state_abbr]}"

\



In [283]:
def get_coordinates_from_wikipedia(page_title):
    base_url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'coordinates|info',
        'titles': page_title
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    # Extract the coordinates and URL from the response
    pages = data['query']['pages']
    page_id = next(iter(pages))
    page_data = pages[page_id]
    coordinates = page_data.get('coordinates', [])
    url = page_data.get('fullurl')

    latitude = coordinates[0]['lat'] if coordinates else None
    longitude = coordinates[0]['lon'] if coordinates else None

    return latitude, longitude, url

# Initialize empty lists for coordinates and URLs
coordinates = []
wiki_urls = []

# Loop over the rows in the DataFrame
for row in full_name_df.itertuples(index=False):
    city = row.Birth_City
    country = row.Birth_Country
    name = row.Name

    # Replace spaces with underscores in the city and country names
    page_title_city = city.replace(' ', '_')
    page_title_country = country.replace(' ', '_')
    page_title_name = name.replace(' ', '_')
    
    # Get coordinates based on the birth city
    latitude, longitude, _ = get_coordinates_from_wikipedia(page_title_city)
    if latitude is None or longitude is None:
        # If coordinates are not found for the birth city, get coordinates based on the birth country
        latitude, longitude, _ = get_coordinates_from_wikipedia(page_title_country)

    url = f'https://en.wikipedia.org/wiki/{page_title_name}'

    coordinates.append((latitude, longitude))
    wiki_urls.append(url)

# Add the coordinates and URLs to the existing DataFrame
full_name_df['Coordinates'] = coordinates
full_name_df['Wikipedia_URL'] = wiki_urls

# Print the updated DataFrame
print(full_name_df)

     Year Laureate_Id                            Name   Category Gender  \
0    1901           1          Wilhelm Conrad Röntgen    physics   male   
1    1902           2              Hendrik A. Lorentz    physics   male   
2    1902           3                   Pieter Zeeman    physics   male   
3    1903           4                 Henri Becquerel    physics   male   
4    1903           5                    Pierre Curie    physics   male   
..    ...         ...                             ...        ...    ...   
984  2022        1019                    Memorial nan      peace    org   
985  2022        1020  Center for Civil Liberties nan      peace    org   
986  2022        1021                    Ben Bernanke  economics   male   
987  2022        1022                 Douglas Diamond  economics   male   
988  2022        1023                   Philip Dybvig  economics   male   

                                            Motivation  Birth_Date  \
0    "in recognition of the e

In [295]:
full_name_df['Coordinates'] = full_name_df['Coordinates'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, tuple) else x)
full_name_df['Wikipedia_URL'] = full_name_df['Wikipedia_URL'].astype(str)

filtered_df = full_name_df.loc[(full_name_df['Gender'] != 'org') & (full_name_df['Coordinates'] != 'None, None')]
filtered_df = filtered_df.reset_index(drop=True)

In [301]:
full_name_df = filtered_df
full_name_df.head()

,Year,Laureate_Id,Name,Category,Gender,Motivation,Birth_Date,Birth_Country,Birth_City,Death_Date,Coordinates,Wikipedia_URL
0,1901,1,Wilhelm Conrad Röntgen,physics,male,"""in recognition of the extraordinary services ...",1845-03-27,Germany,Remscheid,10/2/1923,"51.18333333, 7.2",https://en.wikipedia.org/wiki/Wilhelm_Conrad_R...
1,1902,2,Hendrik A. Lorentz,physics,male,"""in recognition of the extraordinary service t...",1853-07-18,Netherlands,Arnhem,4/2/1928,"51.98333333, 5.91666667",https://en.wikipedia.org/wiki/Hendrik_A._Lorentz
2,1902,3,Pieter Zeeman,physics,male,"""in recognition of the extraordinary service t...",1865-05-25,Netherlands,Zonnemaire,9/10/1943,"51.7125, 3.95083333",https://en.wikipedia.org/wiki/Pieter_Zeeman
3,1903,4,Henri Becquerel,physics,male,"""in recognition of the extraordinary services ...",1852-12-15,France,Paris,25-08-1908,"48.85666667, 2.35222222",https://en.wikipedia.org/wiki/Henri_Becquerel
4,1903,5,Pierre Curie,physics,male,"""in recognition of the extraordinary services ...",1859-05-15,France,Paris,19-04-1906,"48.85666667, 2.35222222",https://en.wikipedia.org/wiki/Pierre_Curie


In [297]:
con = sqlite3.connect("data/nobel_prize.db")
curs = con.cursor()

curs.execute("DROP TABLE IF EXISTS nobel_winners")

curs.execute("""CREATE TABLE recipients (
    Year INTEGER,
    id INTEGER PRIMARY KEY,
    Laureate_Id,
    Name TEXT,
    Category TEXT,
    Gender TEXT,
    Motivation TEXT,
    Birth_Date TEXT,
    Birth_Country TEXT,
    Birth_City TEXT,
    Death_Date TEXT,
    Coordinates TEXT,
    Wikipedia_URL TEXT
    
    )
""")
full_name_df.to_sql("recipients", con, if_exists="append", index=False)


con.commit()
con.close()

